<a href = 'https://ktmt.github.io/blog/2013/09/10/su-dung-metaclass-trong-python/'>Meta class</a>

Giả sử chúng ta có B là metaclass của A. Việc tạo ra một instance của A chính là việc gọi hàm **`A(*args, **kwargs)`**. Nhưng vì A là một instance của B, nên gọi A, chính là gọi hàm **`__call__`** của một instance của B. Do đó, để can thiệp vào quá trình tạo instance của class, chúng ta có thể override hàm **`__call__`** trong class B

**`obj`** is an instance of **`class`**  
**`class`** is an instance of **`metaclass`**

# What is metaclass?

A metaclass is a class whose **instances are classes**.  
Like an "ordinary" class defines the behavior of the instances of the class, a metaclass defines the behavior of classes and their instances.

# Defining metaclass

metaclass inherit from **`type`**

Now we create a very simple metaclass. It's good for nothing, except that it will print the content of its arguments in the __new__ method and returns the results of the type.__new__ call:`

In [12]:
class Meta(type):
    def __new__(cls, classname, superclass, attr_dict):
        
        print('class:',cls)
        print('classname:',classname)
        print('superclass:',superclass)
        print('attribute dict:', attr_dict)
        #this return a class (class creation)
        return type.__new__(cls, classname, superclass, attr_dict)

In [13]:
class S:
    pass
class A(S, metaclass = Meta):
    pass
a = A()

class: <class '__main__.Meta'>
classname: A
superclass: (<class '__main__.S'>,)
attribute dict: {'__module__': '__main__', '__qualname__': 'A'}


after the class definition has been processed, Python calls

`type(classname, superclasses, attributes_dict)`  

When we call "type", the call method of type is called.  
The call method runs two other methods: new and init:  
`type.__new__(typeclass, classname, superclasses, attributedict)`

`type.__init__(cls, classname, superclasses, attributedict)`

This is not the case, if metaclass is declared in the header  
**`Meta.__new__`** and **`Meta.__init__`** will be called

# Example

let's create a class that its definition depends on a value <br>let's say password  if the password is correct, we display the message: welcome, otherwise say: error

In [24]:
password = input('Please enter the password:')


Please enter the password:entrep


In [25]:
class Meta(type):
    def __init__(self, name, supernames, attr_dict):
        if password == 'Trung':
            self.message = lambda self: print('Welcome')
        else:
            self.message = lambda self: print('Error')

In [26]:
class p1(metaclass = Meta):
    pass
class p2(metaclass = Meta):
    pass
class p3(metaclass = Meta):
    pass

In [27]:
trung = p1()
kien = p2()
tu = p3()

In [28]:
trung.message()

Error


In [29]:
kien.message()

Error


# The 'Count Call' metaclass

Create a metaclass, that every class inherits from it will have methods automatically decorated with the call counter function

In [7]:
from functools import wraps
class Meta(type):
    def __new__(cls, classname, superclass, attr_dict):
        pass
    @staticmethod
    def counter(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            wrapper.count += 1
            return func(*args, **kwargs)
        wrapper.count = 0
        return wrapper
    def __new__(cls, classname, superclass, attr_dict):
        for attr in attr_dict:
            if not attr.startswith('__') and callable(attr_dict[attr]):
                attr_dict[attr] = cls.counter(attr_dict[attr])
        return type.__new__(cls, classname, superclass, attr_dict)
    
    
class A(metaclass = Meta):
    def __init__(self):
        pass
    def greetings(self):
        pass
    def say_hi(self):
        pass

In [8]:
a = A()
a.greetings()
a.greetings()
a.greetings()
a.greetings.count

3

In [9]:
a.say_hi()
a.say_hi()
a.say_hi.count

2

## NOTE

instead of calling:
```python
class A(metaclass = Meta):
    pass
```
we can use:  
```python
class A:
    __metaclass__ = Meta
    pass
```